<a href="https://colab.research.google.com/github/KarinaRmzG/Neuronal-Networks/blob/main/CNN_PerrosyGatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def prepare_data():
    """
    Descarga y prepara los datos descargando un archivo ZIP y extrayendo su contenido.

    Returns:
        carpeta_zip (str): La ruta al archivo ZIP descargado y extraído.
    """
    print("Descargando ZIP de datos")

    # Especifica la URL del archivo ZIP de datos.
    url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

    # Utiliza la función get_file de TensorFlow para descargar el archivo ZIP y extraer su contenido.
    carpeta_zip = tf.keras.utils.get_file('cats_and_dogs_filtered.zip', origin=url, extract=True)

    return carpeta_zip

In [4]:
def definir_rutas(carpeta_zip):
    """
    Define las rutas a los directorios relevantes a partir de la ruta del archivo ZIP descargado.

    Args:
        carpeta_zip (str): La ruta al archivo ZIP descargado que contiene los datos.

    Returns:
        carpeta_entrenamiento (str): La ruta al directorio de entrenamiento.
        carpeta_validacion (str): La ruta al directorio de validación.
        carp_entren_gatos (str): La ruta al directorio de imágenes de entrenamiento de gatos.
        carpeta_entren_perros (str): La ruta al directorio de imágenes de entrenamiento de perros.
        carpeta_val_gatos (str): La ruta al directorio de imágenes de validación de gatos.
        carpeta_val_perros (str): La ruta al directorio de imágenes de validación de perros.
    """
    # Obtiene la carpeta base donde se extrajeron los datos del archivo ZIP.
    carpeta_base = os.path.join(os.path.dirname(carpeta_zip), 'cats_and_dogs_filtered')

    # Define las rutas a los directorios relevantes dentro de la estructura de carpetas.
    carpeta_entrenamiento = os.path.join(carpeta_base, 'train')
    carpeta_validacion = os.path.join(carpeta_base, 'validation')
    carp_entren_gatos = os.path.join(carpeta_entrenamiento, 'cats')
    carpeta_entren_perros = os.path.join(carpeta_entrenamiento, 'dogs')
    carpeta_val_gatos = os.path.join(carpeta_validacion, 'cats')
    carpeta_val_perros = os.path.join(carpeta_validacion, 'dogs')

    return carpeta_entrenamiento, carpeta_validacion, carp_entren_gatos, carpeta_entren_perros,carpeta_val_gatos, carpeta_val_perros

In [5]:
def get_data(carp_entren_gatos, carpeta_entren_perros, carpeta_val_gatos, carpeta_val_perros):
    """
    Calcula y devuelve el número total de ejemplos en los conjuntos de datos de entrenamiento y validación.

    Args:
        carp_entren_gatos (str): La ruta al directorio que contiene imágenes de entrenamiento de gatos.
        carpeta_entren_perros (str): La ruta al directorio que contiene imágenes de entrenamiento de perros.
        carpeta_val_gatos (str): La ruta al directorio que contiene imágenes de validación de gatos.
        carpeta_val_perros (str): La ruta al directorio que contiene imágenes de validación de perros.

    Returns:
        total_entrenamiento (int): El número total de ejemplos en el conjunto de datos de entrenamiento.
        total_val (int): El número total de ejemplos en el conjunto de datos de validación.
    """
    # Calcula el número de ejemplos en cada conjunto de datos.
    num_gatos_entren = len(os.listdir(carp_entren_gatos))
    num_perros_entren = len(os.listdir(carpeta_entren_perros))
    num_gatos_val = len(os.listdir(carpeta_val_gatos))
    num_perros_val = len(os.listdir(carpeta_val_perros))

    # Calcula el número total de ejemplos en los conjuntos de datos de entrenamiento y validación.
    total_entrenamiento = num_gatos_entren + num_perros_entren
    total_val = num_gatos_val + num_perros_val

    return total_entrenamiento, total_val

In [6]:
def data_augmentation():
    """
    Realiza aumento de datos mediante la configuración de un generador de datos de entrenamiento.

    Returns:
        image_gen_entrenamiento: Un generador de datos de entrenamiento configurado con técnicas de aumento de datos.
    """
    print("Realizando aumento de datos")

    # Crea un objeto ImageDataGenerator para aplicar técnicas de aumento de datos.
    image_gen_entrenamiento = ImageDataGenerator(
        rescale=1./255,  # Normaliza los valores de píxeles en el rango [0, 1]
        rotation_range=40,  # Rango de rotación en grados
        width_shift_range=0.2,  # Rango de cambio de ancho
        height_shift_range=0.2,  # Rango de cambio de altura
        shear_range=0.2,  # Rango de deformación (cizalla)
        zoom_range=0.2,  # Rango de aumento (zoom)
        horizontal_flip=True,  # Volteo horizontal aleatorio
        fill_mode='nearest'  # Modo de llenado para áreas recién creadas
    )

    return image_gen_entrenamiento

In [7]:
def generate_training_data(image_gen_entrenamiento, carpeta_entrenamiento, TAMANO_LOTE, TAMANO_IMG):
    """
    Genera un generador de datos de entrenamiento a partir de un directorio de imágenes de entrenamiento.

    Args:
        image_gen_entrenamiento (tf.keras.preprocessing.image.ImageDataGenerator): El generador de datos de entrenamiento.
        carpeta_entrenamiento (str): La ruta al directorio que contiene las imágenes de entrenamiento.
        TAMANO_LOTE (int): El tamaño del lote utilizado para cargar imágenes durante el entrenamiento.
        TAMANO_IMG (int): El tamaño al que se redimensionan las imágenes de entrenamiento.

    Returns:
        data_gen_entrenamiento: Un generador de datos de entrenamiento.
    """
    # Utiliza el generador de datos de entrenamiento para cargar y preprocesar las imágenes de entrenamiento.
    data_gen_entrenamiento = image_gen_entrenamiento.flow_from_directory(
        batch_size=TAMANO_LOTE,  # Tamaño del lote de imágenes
        directory=carpeta_entrenamiento,  # Directorio que contiene las imágenes de entrenamiento
        shuffle=True,  # Baraja el orden de las imágenes de entrenamiento
        target_size=(TAMANO_IMG, TAMANO_IMG),  # Tamaño al que se redimensionan las imágenes
        class_mode='binary'  # Modo de clasificación, en este caso, clasificación binaria
    )

    return data_gen_entrenamiento

In [8]:
def generate_validation_data(carpeta_validacion, TAMANO_LOTE, TAMANO_IMG):
    """
    Genera un generador de datos de validación a partir de un directorio de imágenes de validación.

    Args:
        carpeta_validacion (str): La ruta al directorio que contiene las imágenes de validación.
        TAMANO_LOTE (int): El tamaño del lote utilizado para cargar imágenes durante la validación.
        TAMANO_IMG (int): El tamaño al que se redimensionan las imágenes de validación.

    Returns:
        data_gen_validacion: Un generador de datos de validación.
    """
    # Crea un objeto ImageDataGenerator para el preprocesamiento de las imágenes de validación.
    image_gen_val = ImageDataGenerator(rescale=1./255)

    # Utiliza el generador de datos de imágenes para cargar y preprocesar las imágenes de validación.
    data_gen_validacion = image_gen_val.flow_from_directory(
        batch_size=TAMANO_LOTE,  # Tamaño del lote de imágenes
        directory=carpeta_validacion,  # Directorio que contiene las imágenes de validación
        target_size=(TAMANO_IMG, TAMANO_IMG),  # Tamaño al que se redimensionan las imágenes
        class_mode='binary'  # Modo de clasificación, en este caso, clasificación binaria
    )

    return data_gen_validacion

In [9]:
def create_model():
    # Crear un modelo secuencial
    model = tf.keras.models.Sequential()

    # Agregar una capa de convolución 2D con 32 filtros, un tamaño de kernel de (3,3) y función de activación ReLU.
    # La capa de entrada espera imágenes de 150x150 píxeles con 3 canales de color (RGB).
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))

    # Agregar una capa de MaxPooling 2D para reducir el tamaño de la representación.
    model.add(tf.keras.layers.MaxPooling2D(2, 2))

    # Agregar otra capa de convolución 2D con 64 filtros y ReLU.
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))

    # Otra capa de MaxPooling 2D.
    model.add(tf.keras.layers.MaxPooling2D(2, 2))

    # Agregar una capa de convolución 2D con 128 filtros y ReLU.
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))

    # Otra capa de MaxPooling 2D.
    model.add(tf.keras.layers.MaxPooling2D(2, 2))

    # Agregar otra capa de convolución 2D con 128 filtros y ReLU.
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))

    # Otra capa de MaxPooling 2D.
    model.add(tf.keras.layers.MaxPooling2D(2, 2))

    # Agregar una capa de Dropout para la regularización, evitando el sobreajuste.
    model.add(tf.keras.layers.Dropout(0.5))

    # Aplanar la salida de las capas anteriores para conectarla a capas densas (fully connected).
    model.add(tf.keras.layers.Flatten())

    # Agregar una capa densa con 512 neuronas y función de activación ReLU.
    model.add(tf.keras.layers.Dense(512, activation='relu'))

    # Agregar la capa de salida con 2 neuronas para la clasificación binaria (perro o gato).
    # No se aplica una función de activación aquí, ya que se utilizará la función de activación softmax
    # en la función de pérdida durante el entrenamiento.
    model.add(tf.keras.layers.Dense(2))

    return model  # Devolver el modelo construido

In [10]:
#Compilar modelos. Usar crossentropy binario ya que tenemos solo 2 opciones (perro o gato)
def compile_model(model):
    # Compilar el modelo con el optimizador 'adam', una función de pérdida de entropía cruzada categórica dispersa
    # (utilizada para clasificación con etiquetas enteras) y la métrica de 'accuracy' para evaluar el rendimiento.
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

In [11]:
def train_model(model, data_gen_entrenamiento, data_gen_validacion, epocas, total_entrenamiento, TAMANO_LOTE, total_val):
    """
    Entrena un modelo de red neuronal utilizando conjuntos de datos de entrenamiento y validación.

    Args:
        model (tf.keras.Model): El modelo de red neuronal que se va a entrenar.
        data_gen_entrenamiento (tf.keras.preprocessing.image.ImageDataGenerator): Generador de datos de entrenamiento.
        data_gen_validacion (tf.keras.preprocessing.image.ImageDataGenerator): Generador de datos de validación.
        epocas (int): El número de épocas de entrenamiento.
        total_entrenamiento (int): El número total de ejemplos de entrenamiento.
        TAMANO_LOTE (int): El tamaño del lote utilizado durante el entrenamiento.
        total_val (int): El número total de ejemplos de validación.

    Returns:
        None
    """
    print("Entrenando modelo...")

    # El método fit_generator se utiliza para entrenar el modelo con generadores de datos.
    history = model.fit_generator(
        data_gen_entrenamiento,  # Generador de datos de entrenamiento
        steps_per_epoch=int(np.ceil(total_entrenamiento / float(TAMANO_LOTE))),  # Pasos por época
        epochs=epocas,  # Número de épocas de entrenamiento
        validation_data=data_gen_validacion,  # Datos de validación
        validation_steps=int(np.ceil(total_val / float(TAMANO_LOTE)))  # Pasos de validación
    )

    print("Modelo entrenado!")

In [12]:
if __name__ == "__main__":
    TAMANO_LOTE = 100
    TAMANO_IMG = 150
    epocas = 60
    carpeta_zip = prepare_data()
    carpeta_entrenamiento, carpeta_validacion, carp_entren_gatos, carpeta_entren_perros, carpeta_val_gatos, carpeta_val_perros = definir_rutas(carpeta_zip)
    total_entrenamiento, total_val = get_data(carp_entren_gatos, carpeta_entren_perros, carpeta_val_gatos, carpeta_val_perros)
    image_gen_entrenamiento = data_augmentation()
    data_gen_entrenamiento = generate_training_data(image_gen_entrenamiento, carpeta_entrenamiento, TAMANO_LOTE, TAMANO_IMG)
    data_gen_validacion = generate_validation_data(carpeta_validacion, TAMANO_LOTE, TAMANO_IMG)
    model = create_model()
    compile_model(model)
    train_model(model, data_gen_entrenamiento, data_gen_validacion, epocas, total_entrenamiento, TAMANO_LOTE, total_val)

Descargando ZIP de datos
68606236/68606236 [==============================] - 0s 0us/step
Realizando aumento de datos
Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Entrenando modelo...


<ipython-input-11-65b753f5ec13>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/60
20/20 [==============================] - 19s 824ms/step - loss: 0.7103 - accuracy: 0.5040 - val_loss: 0.6904 - val_accuracy: 0.5000
Epoch 2/60
20/20 [==============================] - 18s 931ms/step - loss: 0.6920 - accuracy: 0.5115 - val_loss: 0.6890 - val_accuracy: 0.6090
Epoch 3/60
20/20 [==============================] - 17s 836ms/step - loss: 0.6831 - accuracy: 0.5615 - val_loss: 0.6468 - val_accuracy: 0.6290
Epoch 4/60
20/20 [==============================] - 18s 874ms/step - loss: 0.6926 - accuracy: 0.5340 - val_loss: 0.6915 - val_accuracy: 0.5230
Epoch 5/60
20/20 [==============================] - 17s 835ms/step - loss: 0.6901 - accuracy: 0.5285 - val_loss: 0.6835 - val_accuracy: 0.5830
Epoch 6/60
20/20 [==============================] - 18s 880ms/step - loss: 0.6830 - accuracy: 0.5670 - val_loss: 0.6581 - val_accuracy: 0.6300
Epoch 7/60
20/20 [==============================] - 17s 838ms/step - loss: 0.6652 - accuracy: 0.5950 - val_loss: 0.6776 - val_accuracy: 0.5480

In [ ]:
#Exportar el modelo en formato h5
model.save('perros-gatos.h5')

In [ ]:
#El equipo es Linux. Listemos el contenido de la carpeta actual para ver que se exporto el modelo
!ls

In [ ]:
#Para convertirlo a tensorflow.js, primero debemos instalar la libreria
!pip install tensorflowjs

In [16]:
#Crear carpeta donde se colocaran los archivos resultantes
!mkdir carpeta_salida

In [ ]:
#Realizar la exportacion a la carpeta de salida
!tensorflowjs_converter --input_format keras perros-gatos.h5 carpeta_salida

In [ ]:
#Confirmar que en la carpeta de salida se hayan generado los archivos. Deben aparecer archivos "bin" y "json"
!ls carpeta_salida

In [ ]:
#Para descargarlos, da clic del lado izquierdo en el icono de la carpeta
#y expande carpeta_salida. En los archivos utiliza los 3 puntos para descargarlos